# SQL Queries
## Workbook contains class that contcts to Postgre and performs simple actions involving inputation of data from retreived by etl.py

## The task is to create the following tables and then input provided data:

### Fact Table

1) songplays - records in log data associated with song plays i.e. records with page NextSong
- columns: songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent

### Dimension Tables

1) users - users in the app
- columns: user_id, first_name, last_name, gender, level

2) songs - songs in music database
- columns: song_id, title, artist_id, year, duration

3) artists - artists in music database
- columns: artist_id, name, location, latitude, longitude

4) time - timestamps of records in songplays broken down into specific units
- columns: start_time, hour, day, week, month, year, weekday

In [1]:
import psycopg2
import re
from etl import *

Folder data is in your CWD!


In [2]:
class Postgres:
    '''Connects to Postres and performs simple actions'''
    
    _tables = {}
    
    def __init__(self, host = 'localhost', port='5432', user='postgres', password='student'):
        '''Creates attributes for connecting to database and performin queries'''
        
        self.host = host
        self.port = port
        self.user = user
        self.password = password
        self.con = None
        self.cur = None
        
    def __enter__(self):
        '''Connects to Postgre upon using contex manager'''
        
        self.con = psycopg2.connect(host=self.host, port=self.port, user=self.user, password=self.password)
        self.cur = self.con.cursor()
        self.con.set_session(autocommit=True)
        return self

    def create_database(self, db='sparkifydb'):
        '''Drops database if exists and creates new one'''
        
        self.cur.execute("DROP DATABASE IF EXISTS " + db + ';')
        self.cur.execute('CREATE DATABASE ' + db + ';')
    
    def connect_to_db(self, db='sparkifydb'):
        '''Connects to database'''
        
        self.con = psycopg2.connect(host=self.host, port=self.port, user=self.user, password=self.password, dbname=db)
        self.cur = self.con.cursor()
        self.con.set_session(autocommit=True)
        print('You have been conected to database: '+ db)
        
    def list_tables(self):
        '''Lists tables in database'''
        
        self.cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' ORDER BY table_name;")
        row = self.cur.fetchone()
        tables = ''
        while row:
            tables = tables + str(row)
            row = self.cur.fetchone()
        tables = re.sub(r"[\(\)]",'',tables)[:-1]
        print('At your current DB you have the following tables: '+tables)
        
    def create_table(self, queries):
        '''Creates table'''
        
        assert type(queries)==list, 'List of queries should be provided; The queries should start with CREATE TABLE'
        for query in queries:
            
            assert 'CREATE TABLE' in query, 'The query should start with CREATE TABLE'
            
            name = query.split('CREATE TABLE ')[1].split(' ')[0]
            self.cur.execute('DROP TABLE IF EXISTS '+name+';')
            self.cur.execute(query)
            
            columns = query.split('(')[1].split(')')[0].split(',')
            columns = [col.split( )[0].strip() for col in columns]
            Postgres._tables[name] = columns
            
            print('Table '+name+' created successfully!')
    
    
    def insert_data(self, table_name, data):
        '''Inserts data to table'''
        
        assert table_name in Postgres._tables, 'Create a table beforehand!'
        assert type(data)== list, 'This should be a list of tuples!'
        
        for d in data:

            insert_query = 'INSERT INTO '+ table_name + ' (' + ', '.join(Postgres._tables[table_name])+') VALUES '+ \
                        '(' + '%s, ' * (len(Postgres._tables[table_name])-1)+'%s) ON CONFLICT \
                        ('+Postgres._tables[table_name][0]+') DO NOTHING;'
            
            self.cur.execute(insert_query, d)
            
    def print_columns(self, table_name):
        '''Prints column names of a table'''
        
        assert table_name in Postgres._tables, 'Create a table beforehand!'
        self.cur.execute('Select * FROM '+table_name+' LIMIT 0')
        colnames = [desc[0] for desc in self.cur.description]
        print(colnames)
    
            
    def print_data(self, table_name):
        '''Prints all data of a table'''
        
        self.cur.execute('SELECT * FROM '+table_name+';')
        row = self.cur.fetchone()
        
        print(table_name)
        print(', '.join(Postgres._tables[table_name]))
        while row:
            print(row)
            row = self.cur.fetchone()
        
    def execute_query(self, query):
        '''Executes a custom query'''
        
        self.cur.execute(query)
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        '''Closes the cursor and conection via contex manager'''
        
        self.cur.close()
        self.con.close()
        print('Conection closed!')

In [3]:
songplays = '''CREATE TABLE songplays 
                (songplay_id INT PRIMARY KEY, start_time TIMESTAMP NOT NULL, user_id INT NOT NULL, level VARCHAR NOT NULL,
                song_id VARCHAR NOT NULL, artist_id VARCHAR NOT NULL, session_id INT NOT NULL, 
                location VARCHAR NOT NULL, user_agent VARCHAR NOT NULL);'''
users = '''CREATE TABLE users 
                (user_id INT PRIMARY KEY, first_name VARCHAR, last_name VARCHAR, gender VARCHAR, level VARCHAR);'''
songs = '''CREATE TABLE songs (song_id VARCHAR PRIMARY KEY, title VARCHAR NOT NULL, artist_id VARCHAR NOT NULL, 
year INT NOT NULL, duration INT NOT NULL)'''
artists = '''CREATE TABLE artists (artist_id VARCHAR PRIMARY KEY, name VARCHAR NOT NULL, location VARCHAR NOT NULL,
latitude FLOAT NOT NULL, longitude FLOAT NOT NULL)'''
time = '''CREATE TABLE time (start_time TIMESTAMP PRIMARY KEY, hour INT, day INT, week INT, month INT,
                year INT, weekday INT)'''

queries = [songplays, users, songs, artists, time]

In [4]:
# created for the sake of testing
songplay_table_create = songplays
user_table_create = users
song_table_create = songs
artist_table_create =artists
time_table_create = time

songplay_table_insert = """INSERT INTO songplays (songplay_id, start_time, user_id, level, song_id, artist_id, session_id, 
location, user_agent) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT (songplay_id) DO NOTHING;"""

user_table_insert = """INSERT INTO users (user_id, first_name, last_name, gender, level) VALUES (%s, %s, %s, %s, %s) 
ON CONFLICT (user_id) DO NOTHING;"""

song_table_insert = """INSERT INTO songs (song_id, title, artist_id, year, duration) VALUES (%s, %s, %s, %s, %s) 
ON CONFLICT (song_id) DO NOTHING;"""

artist_table_insert = """'INSERT INTO artists (artist_id, name, location, latitude, longitude) VALUES (%s, %s, %s, %s, %s) 
ON CONFLICT (artist_id) DO NOTHING;';"""


time_table_insert = '''INSERT INTO time (start_time, hour, day, week, month, year, weekday) 
VALUES (%s, %s, %s, %s, %s, %s, %s) ON CONFLICT (start_time) DO NOTHING;'''

In [5]:
if __name__ =='__main__':
    with Postgres() as p:
        p.create_database()
        p.connect_to_db()
        print(p._tables)
        p.create_table(queries)
        p.list_tables()
        for name in ['songplays', 'users', 'songs', 'artists', 'time']:
            p.print_columns(name)
            print('-------------')
        p.insert_data('users', users_data)
        p.print_data('users')
        print('------------------')
        p.insert_data('songs', songs_data)
        p.print_data('songs')
        print('------------------')
        p.insert_data('artists', artists_data)
        p.print_data('artists')
        print('------------------')
        p.insert_data('time', time_data)
        p.print_data('time')
        print('------------------')
        p.insert_data('songplays', songplays_data)
        p.print_data('songplays')

You have been conected to database: sparkifydb
{}
Table songplays created successfully!
Table users created successfully!
Table songs created successfully!
Table artists created successfully!
Table time created successfully!
At your current DB you have the following tables: 'artists','songplays','songs','time','users'
['songplay_id', 'start_time', 'user_id', 'level', 'song_id', 'artist_id', 'session_id', 'location', 'user_agent']
-------------
['user_id', 'first_name', 'last_name', 'gender', 'level']
-------------
['song_id', 'title', 'artist_id', 'year', 'duration']
-------------
['artist_id', 'name', 'location', 'latitude', 'longitude']
-------------
['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']
-------------
users
user_id, first_name, last_name, gender, level
(39, 'Walter', 'Frye', 'M', 'free')
(8, 'Kaylee', 'Summers', 'F', 'free')
(10, 'Sylvie', 'Cruz', 'F', 'free')
(26, 'Ryan', 'Smith', 'M', 'free')
(101, 'Jayden', 'Fox', 'M', 'free')
(83, 'Stefany', 'White', '

time
start_time, hour, day, week, month, year, weekday
(datetime.datetime(2018, 11, 1, 20, 57, 10, 796000), 20, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 1, 46, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 2, 12, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 5, 52, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 8, 16, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 11, 13, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 17, 33, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 24, 53, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 28, 54, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 42, 0, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 50, 15, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11, 1, 21, 52, 5, 796000), 21, 1, 44, 11, 2018, 3)
(datetime.datetime(2018, 11

(datetime.datetime(2018, 11, 14, 7, 12, 32, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 14, 25, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 16, 10, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 18, 17, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 21, 47, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 25, 12, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 25, 54, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 29, 17, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 29, 33, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 32, 16, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 32, 40, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 35, 40, 796000), 7, 14, 46, 11, 2018, 2)
(datetime.datetime(2018, 11, 14, 7, 36, 21, 796000), 7, 14, 46, 11, 2018, 2)

(datetime.datetime(2018, 11, 20, 15, 26, 53, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 30, 30, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 35, 10, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 40, 7, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 40, 56, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 44, 23, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 44, 43, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 48, 5, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 48, 12, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 50, 1, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 51, 9, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 53, 17, 796000), 15, 20, 47, 11, 2018, 1)
(datetime.datetime(2018, 11, 20, 15, 54, 31, 796000), 15

(datetime.datetime(2018, 11, 29, 21, 14, 16, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 14, 49, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 16, 53, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 17, 30, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 19, 18, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 20, 57, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 21, 47, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 22, 18, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 24, 3, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 24, 35, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 27, 0, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 27, 48, 796000), 21, 29, 48, 11, 2018, 3)
(datetime.datetime(2018, 11, 29, 21, 27, 50, 796000), 

songplays
songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
(0, datetime.datetime(2018, 11, 1, 20, 57, 10, 796000), 39, 'free', 'NaN', 'NaN', 38, 'San Francisco-Oakland-Hayward, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(1, datetime.datetime(2018, 11, 1, 21, 1, 46, 796000), 8, 'free', 'NaN', 'NaN', 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
(2, datetime.datetime(2018, 11, 1, 21, 2, 12, 796000), 8, 'free', 'NaN', 'NaN', 139, 'Phoenix-Mesa-Scottsdale, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36"')
(3, datetime.datetime(2018, 11, 1, 21, 50, 15, 796000), 26, 'free', 'NaN', 'NaN', 169, 'San Jose-Sunnyvale-Santa Clara, CA', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML,

(2459, datetime.datetime(2018, 11, 5, 16, 8, 52, 796000), 24, 'paid', 'NaN', 'NaN', 23, 'Lake Havasu City-Kingman, AZ', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
(2460, datetime.datetime(2018, 11, 5, 16, 10, 31, 796000), 73, 'paid', 'NaN', 'NaN', 255, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(2461, datetime.datetime(2018, 11, 5, 16, 12, 35, 796000), 97, 'paid', 'NaN', 'NaN', 147, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
(2462, datetime.datetime(2018, 11, 5, 16, 15, 10, 796000), 73, 'paid', 'NaN', 'NaN', 255, 'Tampa-St. Petersburg-Clearwater, FL', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
(2463, datetime.datetime(2018, 11

(4567, datetime.datetime(2018, 11, 14, 15, 42, 26, 796000), 25, 'paid', 'NaN', 'NaN', 534, 'Marinette, WI-MI', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(4568, datetime.datetime(2018, 11, 15, 12, 38, 3, 796000), 80, 'paid', 'NaN', 'NaN', 611, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(4569, datetime.datetime(2018, 11, 14, 15, 43, 22, 796000), 80, 'paid', 'NaN', 'NaN', 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(4570, datetime.datetime(2018, 11, 14, 15, 45, 40, 796000), 25, 'paid', 'NaN', 'NaN', 534, 'Marinette, WI-MI', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(4571, datetime.datetime(2018, 11, 14, 15, 47, 46, 79

(6943, datetime.datetime(2018, 11, 24, 18, 41, 11, 796000), 49, 'paid', 'NaN', 'NaN', 849, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(6944, datetime.datetime(2018, 11, 24, 18, 45, 3, 796000), 49, 'paid', 'NaN', 'NaN', 849, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(6945, datetime.datetime(2018, 11, 24, 18, 48, 40, 796000), 49, 'paid', 'NaN', 'NaN', 849, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(6946, datetime.datetime(2018, 11, 24, 18, 52, 22, 796000), 49, 'paid', 'NaN', 'NaN', 849, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
(6947, datetime.datetime(2018, 11, 24, 18, 56, 31, 796000), 86, 'free', 'NaN', 'NaN', 906, 'La Crosse-Onalaska, WI-MN', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrom